<center>
    <h1> INF477 - Redes Neuronales Artificiales  </h1>
    <h2> Tarea #3 - Parte I: Entrenamiento de RNNs en una serie de tiempo </h2> 
</center>

Tarea realizada por:
* Alvaro Salinas - `alvaro.salinase@gmail.com` - **rol:** 201073001-8
* Martín Villanueva - `martin.villanueva@alumnos.usm.cl` - **rol:** 201104012-0

_DI UTFSM. Noviembre 2016._

## Tabla de Contenidos
* [Parte (a)](#a)
* [Parte (b)](#b)
* [Parte (c)](#c)
* [Parte (d)](#d)
* [Parte (e)](#e)
* [Parte (f)](#f)
* [Parte (g)](#g)
* [Parte (h)](#h)
* [Parte (i)](#i)
* [Parte (j)](#j)
* [Parte (k)](#k)
* [Parte (l)](#l)
* [Parte (m)](#m)